In [117]:
import re
import pandas as pd
import itertools
import random
import json
from collections import Counter


In [118]:
dataset = []
with open('../data/ViMedNER.txt', 'r', encoding='utf-8') as f:
    for dong in f:
        dataset.append(json.loads(dong.strip()))

In [119]:
df = pd.DataFrame(dataset)
df.head()

,id,text,label,Comments
0,3504,kháng sinh dự phòng nếu đã được chỉ định trong...,"[[135, 163, DISEASE]]",[]
1,3505,ngày nay các chuyên gia tin rằng viêm nội tâm ...,"[[33, 49, DISEASE], [101, 109, CAUSE]]",[]
2,3506,hướng dẫn hiện nay khuyến cáo điều trị kháng s...,"[[129, 157, DISEASE]]",[]
3,3507,bác sĩ vẫn có thể khuyên nên dùng kháng sinh p...,"[[84, 100, TREATMENT]]",[]
4,3508,đối với hầu hết những người có thông liên thất...,"[[31, 46, DISEASE], [160, 176, DISEASE]]",[]


In [120]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7749 entries, 0 to 7748
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7749 non-null   int64 
 1   text      7749 non-null   object
 2   label     7749 non-null   object
 3   Comments  7749 non-null   object
dtypes: int64(1), object(3)
memory usage: 242.3+ KB


In [121]:
print(df['Comments'].value_counts())
df = df.drop(columns='Comments')

Comments
[]    7749
Name: count, dtype: int64


In [122]:
df_exploded = df['label'].explode()
label_counts = df_exploded.dropna().apply(lambda x: x[2]).value_counts()
print(label_counts)

label
DISEASE       9712
SYMPTOM       4195
TREATMENT     3084
DIAGNOSTIC    1559
CAUSE         1433
Name: count, dtype: int64


In [123]:
TARGET_RELATIONS = {
    frozenset(['DISEASE', 'SYMPTOM']),
    frozenset(['DISEASE', 'TREATMENT']),
    frozenset(['CAUSE', 'DISEASE']),
    frozenset(['DIAGNOSTIC', 'DISEASE'])
}

def count_relations_with_ratio(data, ratio=100):
    positive_counter = Counter()
    negative_counter = Counter()
    
    total_pos = 0
    total_neg = 0

    for entry in data:
        # LƯU Ý QUAN TRỌNG: 
        # Bạn nên dùng list thay vì set. Nếu dùng set, các entity cùng loại (VD: 2 bệnh trong 1 câu) 
        # sẽ bị gộp thành 1, dẫn đến mất cặp Negative "DISEASE - DISEASE".
        entity_types = [item[2] for item in entry['label']]
        
        if len(entity_types) < 2:
            continue

        # Tạo tất cả các cặp
        pairs = list(itertools.combinations(entity_types, 2))
        
        sent_positives = []
        sent_negatives = []

        # Phân loại
        for p in pairs:
            p_set = frozenset(p)
            p_str = f"{sorted(p)[0]} - {sorted(p)[1]}"

            if p_set in TARGET_RELATIONS:
                sent_positives.append(p_str)
            else:
                sent_negatives.append(p_str)

        # --- [ĐÃ SỬA] XỬ LÝ LOGIC LẤY MẪU ---
        num_pos = len(sent_positives)
        
        if num_pos == 0:
            # TRƯỜNG HỢP 1: Không có Positive nào -> Lấy TẤT CẢ Negative tìm được
            valid_negatives = sent_negatives
        else:
            max_neg = num_pos * ratio
            valid_negatives = sent_negatives[:max_neg]

        # --- Cập nhật Count ---
        positive_counter.update(sent_positives)
        negative_counter.update(valid_negatives)
        
        total_pos += num_pos
        total_neg += len(valid_negatives)

    return positive_counter, negative_counter, total_pos, total_neg

pos_counts, neg_counts, t_pos, t_neg = count_relations_with_ratio(dataset)

print("=== KẾT QUẢ ĐẾM (POSITIVE RELATIONS) ===")
for pair, count in pos_counts.most_common():
    print(f"{pair}: {count}")
print(f"-> Tổng cộng Positive: {t_pos}")

print("\n=== KẾT QUẢ ĐẾM (NO-RELATION / NEGATIVE) ===")
if not neg_counts:
    print("Không có cặp No-Relation nào được chọn.")
else:
    for pair, count in neg_counts.most_common():
        print(f"{pair}: {count}")
print(f"-> Tổng cộng No-Relation: {t_neg}")

=== KẾT QUẢ ĐẾM (POSITIVE RELATIONS) ===
DISEASE - SYMPTOM: 6162
DISEASE - TREATMENT: 2418
DIAGNOSTIC - DISEASE: 2029
CAUSE - DISEASE: 1926
-> Tổng cộng Positive: 12535

=== KẾT QUẢ ĐẾM (NO-RELATION / NEGATIVE) ===
DISEASE - DISEASE: 10640
SYMPTOM - SYMPTOM: 7614
TREATMENT - TREATMENT: 3116
DIAGNOSTIC - DIAGNOSTIC: 1493
CAUSE - CAUSE: 1277
SYMPTOM - TREATMENT: 754
DIAGNOSTIC - SYMPTOM: 547
CAUSE - SYMPTOM: 527
DIAGNOSTIC - TREATMENT: 238
CAUSE - TREATMENT: 180
CAUSE - DIAGNOSTIC: 68
-> Tổng cộng No-Relation: 26454


In [124]:
RELATION_LABELS = [
    "NO_RELATION",          # Nhãn 0: Không có quan hệ
    "CAUSE_DISEASE",        # Nhãn 1: Nguyên nhân - Bệnh (C đứng trước D)
    "DIAGNOSTIC_DISEASE",   # Nhãn 2: Chẩn đoán - Bệnh (Dia... trước Dis...)
    "DISEASE_SYMPTOM",      # Nhãn 3: Bệnh - Triệu chứng (D đứng trước S)
    "DISEASE_TREATMENT"     # Nhãn 4: Bệnh - Điều trị (D đứng trước T)
]

In [125]:
# Từ Nhãn sang Số (Dùng khi tạo dataset)
label2id = {
    "NO_RELATION": 0,
    "CAUSE_DISEASE": 1,
    "DIAGNOSTIC_DISEASE": 2,
    "DISEASE_SYMPTOM": 3,
    "DISEASE_TREATMENT": 4
}

# Từ Số sang Nhãn (Dùng khi predict/infer)
id2label = {
    0: "NO_RELATION",
    1: "CAUSE_DISEASE",
    2: "DIAGNOSTIC_DISEASE",
    3: "DISEASE_SYMPTOM",
    4: "DISEASE_TREATMENT"
}

In [126]:
TARGET_RELATIONS = {
    frozenset(['DISEASE', 'SYMPTOM']),
    frozenset(['DISEASE', 'TREATMENT']),
    frozenset(['CAUSE', 'DISEASE']),
    frozenset(['DIAGNOSTIC', 'DISEASE'])
}

def generate_relation_dataset(data, ratio=100):
    final_dataset = []

    for entry in data:
        text = entry['text']
        original_labels = entry['label'] # [[start, end, "TYPE"], ...]
        
        # 1. Parse thực thể & Lấy text tương ứng
        # Lưu thành list các dict để dễ truy xuất sau này
        entities = []
        for start, end, label_type in original_labels:
            entity_text = text[start:end]
            entities.append({
                "text": entity_text,
                "type": label_type,
                "start": start,
                "end": end
            })

        # Nếu câu có ít hơn 2 thực thể -> Bỏ qua
        if len(entities) < 2:
            continue

        # 2. Tạo tổ hợp cặp (Combinations)
        # pairs sẽ chứa các cặp entity objects
        pairs = list(itertools.combinations(entities, 2))
        
        # Danh sách tạm để chứa các cặp tìm được trong câu này
        temp_positives = []
        temp_negatives = []

        for e1, e2 in pairs:
            # Xác định loại quan hệ
            pair_types = frozenset([e1['type'], e2['type']])
            
            # Tạo record cho dataset
            # Sắp xếp type để Label luôn nhất quán (VD: luôn là DISEASE_SYMPTOM)
            sorted_types = sorted([e1['type'], e2['type']])
            relation_key = f"{sorted_types[0]}_{sorted_types[1]}"
            
            record = {
                "text": text,
                "relation": None, 
                "entity_1": {
                    "text": e1['text'],
                    "type": e1['type'],
                    "start": e1['start'],
                    "end": e1['end']
                },
                "entity_2": {
                    "text": e2['text'],
                    "type": e2['type'],
                    "start": e2['start'],
                    "end": e2['end']
                }
            }
            if pair_types in TARGET_RELATIONS:
                record['relation'] = relation_key # Ví dụ: "DISEASE_SYMPTOM"
                # Map sang ID luôn nếu cần: record['label_id'] = label2id[relation_key]
                temp_positives.append(record)
            else:
                record['relation'] = "NO_RELATION"
                # Map sang ID luôn nếu cần: record['label_id'] = label2id["NO_RELATION"]
                temp_negatives.append(record)

        # 3. LOGIC LỌC DỮ LIỆU (QUAN TRỌNG)
        num_pos = len(temp_positives)
        
        selected_negatives = []
        
        if num_pos == 0:
            # TRƯỜNG HỢP A: Không có Positive -> Lấy HẾT Negative
            selected_negatives = temp_negatives
        else:
            # TRƯỜNG HỢP B: Có Positive -> Lấy Negative theo tỷ lệ
            # Xáo trộn ngẫu nhiên trước khi cắt để khách quan (optional)
            random.shuffle(temp_negatives)
            
            max_neg = num_pos * ratio
            selected_negatives = temp_negatives[:max_neg]

        # 4. Tổng hợp vào dataset chính
        final_dataset.extend(temp_positives)
        final_dataset.extend(selected_negatives)

    return final_dataset

In [127]:
training_data = generate_relation_dataset(dataset, ratio=100)

print(f"Tổng số mẫu dữ liệu tạo ra: {len(training_data)}")
print("-" * 50)

output_filename = '../data/re_medical_dataset.json'
with open(output_filename, 'w', encoding='utf-8') as f:
    json.dump(
        training_data, 
        f, 
        ensure_ascii=False, 
        indent=4             
    )

print(f"Đã lưu xong {len(training_data)} mẫu dữ liệu vào file '{output_filename}'")

Tổng số mẫu dữ liệu tạo ra: 38989
--------------------------------------------------
Đã lưu xong 38989 mẫu dữ liệu vào file '../data/re_medical_dataset.json'


In [128]:
def print_dataset_statistics(original_data, final_dataset):
    print("="*60)
    print("BÁO CÁO THỐNG KÊ DỮ LIỆU (DATASET STATISTICS)")
    print("="*60)

    # --- 1. THỐNG KÊ ENTITY (Từ dữ liệu gốc) ---
    entity_counter = Counter()
    total_entities = 0
    
    for entry in original_data:
        # entry['label'] cấu trúc: [[start, end, "TYPE"], ...]
        for _, _, label_type in entry['label']:
            entity_counter[label_type] += 1
            total_entities += 1
            
    print(f"\n1️) THỐNG KÊ THỰC THỂ (ENTITIES) - Tổng số: {total_entities}")
    print(f"{'-'*60}")
    print(f"{'Entity Type':<25} | {'Count':<10} | {'Percentage':<10}")
    print(f"{'-'*60}")
    
    for entity, count in entity_counter.most_common():
        percent = (count / total_entities) * 100
        print(f"{entity:<25} | {count:<10} | {percent:.2f}%")

    # --- 2. THỐNG KÊ RELATION (Từ dữ liệu training đã sinh) ---
    relation_counter = Counter()
    total_relations = len(final_dataset)
    
    for item in final_dataset:
        relation_counter[item['relation']] += 1
        
    print(f"\n2️)  THỐNG KÊ QUAN HỆ (RELATIONS) - Tổng số mẫu: {total_relations}")
    print(f"{'-'*60}")
    print(f"{'Relation Label':<25} | {'Count':<10} | {'Percentage':<10}")
    print(f"{'-'*60}")
    
    # Tách Positive và Negative để tính tỷ lệ thực tế
    total_pos = 0
    total_neg = 0
    
    for rel, count in relation_counter.most_common():
        percent = (count / total_relations) * 100
        print(f"{rel:<25} | {count:<10} | {percent:.2f}%")
        
        if rel == "NO_RELATION":
            total_neg += count
        else:
            total_pos += count

    # --- 3. TỔNG KẾT TỶ LỆ ---
    print(f"\n3️)  TỔNG KẾT TỶ LỆ POSITIVE / NEGATIVE")
    print(f"{'-'*60}")
    if total_pos > 0:
        actual_ratio = total_neg / total_pos
        print(f"Positive Samples: {total_pos}")
        print(f"Negative Samples: {total_neg}")
        print(f"Tỷ lệ thực tế: 1 Positive : {actual_ratio:.1f} Negative")
    else:
        print("Cảnh báo: Không có mẫu Positive nào!")
    print("="*60)

# --- CHẠY THỬ VỚI DỮ LIỆU Ở BƯỚC TRƯỚC ---
# data_sample: List gốc ban đầu
# training_data: List kết quả sau khi chạy generate_relation_dataset

print_dataset_statistics(dataset, training_data)

BÁO CÁO THỐNG KÊ DỮ LIỆU (DATASET STATISTICS)

1️) THỐNG KÊ THỰC THỂ (ENTITIES) - Tổng số: 19983
------------------------------------------------------------
Entity Type               | Count      | Percentage
------------------------------------------------------------
DISEASE                   | 9712       | 48.60%
SYMPTOM                   | 4195       | 20.99%
TREATMENT                 | 3084       | 15.43%
DIAGNOSTIC                | 1559       | 7.80%
CAUSE                     | 1433       | 7.17%

2️)  THỐNG KÊ QUAN HỆ (RELATIONS) - Tổng số mẫu: 38989
------------------------------------------------------------
Relation Label            | Count      | Percentage
------------------------------------------------------------
NO_RELATION               | 26454      | 67.85%
DISEASE_SYMPTOM           | 6162       | 15.80%
DISEASE_TREATMENT         | 2418       | 6.20%
DIAGNOSTIC_DISEASE        | 2029       | 5.20%
CAUSE_DISEASE             | 1926       | 4.94%

3️)  TỔNG KẾT TỶ LỆ POS